In [1]:
import pandas as pd
import numpy as np

In [2]:
movies=pd.read_csv("movies_metadata.csv")
ratings=pd.read_csv("ratings_small.csv")

In [3]:
movies=movies[['title','id']]
movies['id']=movies['id'].astype(int)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8965 entries, 0 to 8964
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   8964 non-null   object
 1   id      8965 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 140.2+ KB


<ipython-input-3-44ce36f159ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['id']=movies['id'].astype(int)


In [4]:
ratings.head()
ratings.info()
movies.rename(columns={"id":"movieId"},inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
#Using correlation method
merged=pd.merge(movies,ratings,on="movieId")
merged.head()

,title,movieId,userId,rating,timestamp
0,Heat,949,23,3.5,1148721092
1,Heat,949,102,4.0,956598942
2,Heat,949,232,2.0,955092697
3,Heat,949,242,5.0,956688825
4,Heat,949,263,3.0,1117846575


In [6]:
average_rating_df=merged.groupby("title").agg(
    avg_rating=('rating','mean'),
    total_count=('title','count')
).reset_index()
average_rating_df.sort_values('total_count',ascending=False).head()

,title,avg_rating,total_count
969,Terminator 3: Rise of the Machines,4.256173,324
1141,The Million Dollar Hotel,4.487138,311
904,Solaris,4.134426,305
971,The 39 Steps,4.221649,291
672,Monsoon Wedding,3.706204,274


# correlation

In [7]:
merged.head()
corr_features=merged[['userId',"title","rating"]]

In [8]:
correlation_matrix=merged.pivot_table(index="userId",columns="title",values="rating")
correlation_matrix.head()

title,...And God Created Woman,10 Things I Hate About You,11'09''01 - September 11,12 Angry Men,15 Minutes,1900,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,21 Grams,...,Z,Zabriskie Point,Zardoz,Zatoichi,Zazie dans le métro,Zombie Flesh Eaters,Zombie Holocaust,eXistenZ,xXx,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
corr_index=np.random.choice(average_rating_df.shape[0])
average_rating_df.iloc[corr_index,:]

title          Dial M for Murder
avg_rating                   3.6
total_count                   10
Name: 289, dtype: object

In [10]:
selected_movie=correlation_matrix['Dial M for Murder']

In [11]:
similar_movies=correlation_matrix.corrwith(selected_movie)
similar_movies

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


title
...And God Created Woman     NaN
10 Things I Hate About You   NaN
11'09''01 - September 11     NaN
12 Angry Men                 NaN
15 Minutes                   NaN
                              ..
Zombie Flesh Eaters          NaN
Zombie Holocaust             NaN
eXistenZ                     NaN
xXx                          NaN
¡Three Amigos!               NaN
Length: 1389, dtype: float64

In [12]:
result=pd.DataFrame(similar_movies,columns=["correlation"]).reset_index()
result.sort_values("correlation",ascending=False).head()

,title,correlation
1086,The House of the Spirits,1.0
966,Teenage Mutant Ninja Turtles III,1.0
1255,Thelma & Louise,1.0
352,Fahrenheit 9/11,1.0
603,Lolita,1.0


In [14]:
final_result=pd.merge(result,average_rating_df[['title','total_count']],on="title")
final_result[final_result['total_count']>100].sort_values("correlation",ascending=False).head()

,title,correlation,total_count
782,Point Break,0.986440,132
31,A Clockwork Orange,0.927173,102
806,Rebecca,0.738549,109
672,Monsoon Wedding,0.644715,274
600,Live and Let Die,0.588069,125


In [23]:
def movie_corr(movie_name,number):
  the_movie=correlation_matrix[movie_name]
  similar_movie=correlation_matrix.corrwith(the_movie)
  result=pd.DataFrame(similar_movies,columns=["correlation"]).reset_index()
  final_result=pd.merge(result,average_rating_df[['title','total_count']],on="title")
  return final_result[final_result['total_count']>100].sort_values("correlation",ascending=False).head(number)


In [76]:
movie_corr("Rebecca",10)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,title,correlation,total_count
782,Point Break,0.986,132
31,A Clockwork Orange,0.927,102
806,Rebecca,0.739,109
672,Monsoon Wedding,0.645,274
600,Live and Let Die,0.588,125
40,A River Runs Through It,0.577,132
645,Men in Black II,0.542,224
1302,Twin Peaks: Fire Walk with Me,0.500,123
839,Rope,0.500,190
382,Fools Rush In,0.500,117


# Knn CLustering for recommendation


In [25]:
pd.set_option("display.float_format",lambda x:"%.3f" %x)
average_rating_df['total_count'].describe()


count   1389.000
mean      20.654
std       37.468
min        1.000
25%        2.000
50%        6.000
75%       22.000
max      324.000
Name: total_count, dtype: float64

In [26]:
rating_count_thresold=20

titles=average_rating_df[average_rating_df['total_count']>rating_count_thresold]['title']


knn_features=merged[merged['title'].isin(titles)]
knn_features.head()

,title,movieId,userId,rating,timestamp
18,Cutthroat Island,1408,7,1.000,851869140
19,Cutthroat Island,1408,11,5.000,1391658667
20,Cutthroat Island,1408,19,3.000,855192910
21,Cutthroat Island,1408,23,3.500,1148720760
22,Cutthroat Island,1408,57,4.000,907763283


In [27]:
knn_matrix=knn_features.pivot_table(index="title",columns="userId",values="rating").fillna(0)
knn_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",0.000,0.000,0.000,3.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
24 Hour Party People,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
28 Days Later,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
48 Hrs.,0.000,5.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,...,4.000,0.000,0.000,0.000,4.000,4.000,0.000,0.000,3.000,0.000


In [28]:
#Array matrix
from scipy.sparse import csr_matrix

array_matrix=csr_matrix(knn_matrix)

In [29]:
from sklearn.neighbors import NearestNeighbors

knn_model= NearestNeighbors(metric="cosine",algorithm="brute")

knn_model.fit(array_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [83]:
knn_index=np.random.choice(knn_matrix.shape[0])
print(knn_index)
print(merged.iloc[knn_index,:])

128
title        Four Rooms
movieId               5
userId              318
rating            3.000
timestamp     862902663
Name: 128, dtype: object


In [84]:
distances,indices=knn_model.kneighbors(knn_matrix.iloc[knn_index,:].values.reshape(1,-1),n_neighbors=6)

In [87]:
distances

array([[0.        , 0.52048878, 0.53533826, 0.53680972, 0.54098603,
        0.54658466]])

In [89]:
for i in range(0,len(distances.flatten())):
  if i==0:
      print("Recommendation for {0} : \n".format(knn_matrix.index[knn_index]))
  else:
      print("{0} : {1} ,with distance of {2} :".format(i,knn_matrix.index[indices.flatten()[i]],distances.flatten()[i]))
      #print("{1}: {2} with distance of {3}".format(i,knn_matrix.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendation for Heathers : 

1 : American Pie ,with distance of 0.5204887791995556 :
2 : Light of Day ,with distance of 0.535338257314743 :
3 : Rope ,with distance of 0.5368097197210515 :
4 : Mr. Holland's Opus ,with distance of 0.5409860329341996 :
5 : Two Weeks Notice ,with distance of 0.54658466429536 :


In [90]:
def knn_recommendation(name_of_movie,number):
  distances,indices=knn_model.kneighbors(knn_matrix.loc[name_of_movie,:].values.reshape(1,-1),n_neighbors=number+1)
  for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendation for {0} : \n".format(name_of_movie))
    else:
        print("{0} : {1} ,with distance of {2} :".format(i,knn_matrix.index[indices.flatten()[i]],distances.flatten()[i]))



In [93]:
knn_recommendation("Light of Day",5)

Recommendation for Light of Day : 

1 : Aguirre: The Wrath of God ,with distance of 0.4505912631783241 :
2 : Sister Act ,with distance of 0.45562132238036734 :
3 : Rope ,with distance of 0.4747014220402441 :
4 : The 39 Steps ,with distance of 0.49200976118049156 :
5 : My Tutor ,with distance of 0.5053774805759237 :
